In [1]:
import open3d as o3d
import sys
import numpy as np
from copy import deepcopy

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
source_pcd = o3d.io.read_point_cloud("./vin-experiments/source_mem_2_1100_80_0.pcd")
target_pcd = o3d.io.read_point_cloud("./vin-experiments/target_mem_822_1700_80_0.pcd")

In [3]:
o3d.visualization.draw_geometries([source_pcd])

In [4]:
bbsize = 5

In [5]:
box = o3d.geometry.TriangleMesh.create_box(width=bbsize, depth=bbsize, height=0.00001)
box.paint_uniform_color([0.0, 0.5, 0.5])

box = box.translate([10, -1, -0.4])

o3d.visualization.draw_geometries([source_pcd, box])

In [6]:
source_sliding_window = []

source_box_centers = [
    [3, -1, 2],
    [5, -1, 2],
    [7, -1, 2],
    [9, -1, 2],
    [11, -1, 2],
    [13, -1, 2],
]

for i in source_box_centers:
    x_min = i[0] - bbsize/2
    x_max = i[0] + bbsize/2
    z_min = i[2] - bbsize/2
    z_max = i[2] + bbsize/2

    # Filter points based on criteria
    filtered_points = []
    for point in source_pcd.points:
        if x_min < point[0] < x_max and z_min < point[2] < z_max:
            filtered_points.append(point)

    # Convert filtered points to a new point cloud
    filtered_point_cloud = o3d.geometry.PointCloud()
    filtered_point_cloud.points = o3d.utility.Vector3dVector(np.array(filtered_points))

    box = o3d.geometry.TriangleMesh.create_box(width=bbsize, depth=bbsize, height=0.00001)
    box.paint_uniform_color([0.0, 0.5, 0.5])

    box = box.translate(i)

    # o3d.visualization.draw_geometries([filtered_point_cloud])

    source_sliding_window.append([filtered_point_cloud, np.array(i)])


In [7]:
target_sliding_window = []

target_box_centers = [
    [9, -1, 2],
    [11, -1, 2],
    [13, -1, 2],
    [13, -1, 4],
    [13, -1, 6]
]

for i in target_box_centers:
    x_min = i[0] - bbsize/2
    x_max = i[0] + bbsize/2
    z_min = i[2] - bbsize/2
    z_max = i[2] + bbsize/2

    # Filter points based on criteria
    filtered_points = []
    for point in target_pcd.points:
        if x_min < point[0] < x_max and z_min < point[2] < z_max:
            filtered_points.append(point)

    # Convert filtered points to a new point cloud
    filtered_point_cloud = o3d.geometry.PointCloud()
    filtered_point_cloud.points = o3d.utility.Vector3dVector(np.array(filtered_points))

    # o3d.visualization.draw_geometries([filtered_point_cloud])

    target_sliding_window.append([filtered_point_cloud, np.array(i)])

In [8]:
def downsample_and_compute_fpfh(pcd, voxel_size):
    # Downsample the point cloud using Voxel Grid
    pcd_down = deepcopy(pcd)

    radius_normal = voxel_size * 2
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh

def register_point_clouds(source, target, voxel_size = 0.05, global_dist_factor = 1.5, local_dist_factor = 0.4):
    source_down, source_fpfh = downsample_and_compute_fpfh(source, voxel_size)
    target_down, target_fpfh = downsample_and_compute_fpfh(target, voxel_size)

    distance_threshold = voxel_size * global_dist_factor
    # print(":: RANSAC registration on downsampled point clouds.")
    # print("   Since the downsampling voxel size is %.3f," % voxel_size)
    # print("   we use a liberal distance threshold %.3f." % distance_threshold)
    result_ransac = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh, True,
        distance_threshold,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        3, [
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(
                0.9),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(
                distance_threshold)
        ], o3d.pipelines.registration.RANSACConvergenceCriteria(4000000, 500))

    # Refine the registration using ICP
    result_icp = o3d.pipelines.registration.registration_icp(
        source_down, target_down, voxel_size*local_dist_factor, result_ransac.transformation,
        o3d.pipelines.registration.TransformationEstimationPointToPoint()
    )

    return result_icp

In [9]:
results = []

for i, (source_slide_pcd, source_slide_pose) in enumerate(source_sliding_window):
    for j, (target_slide_pcd, target_slide_pose) in enumerate(target_sliding_window):

        cur_source_slide_pcd = deepcopy(source_slide_pcd)
        cur_target_slide_pcd = deepcopy(target_slide_pcd)

        # o3d.visualization.draw_geometries([cur_source_slide_pcd, cur_target_slide_pcd])

        # mean centering
        cur_source_slide_pcd = cur_source_slide_pcd.translate(-source_slide_pose)
        cur_target_slide_pcd = cur_target_slide_pcd.translate(-target_slide_pose)

        # registration

        # o3d.visualization.draw_geometries([cur_source_slide_pcd, cur_target_slide_pcd])

        result = register_point_clouds(cur_source_slide_pcd, cur_target_slide_pcd)

        print(f"{i} {j}:\t Inlier RMSE - {result.inlier_rmse}\t Fitness: {result.fitness}")

        cur_target_slide_pcd = cur_target_slide_pcd.transform(result.transformation)

        # o3d.visualization.draw_geometries([cur_source_slide_pcd, cur_target_slide_pcd])

        results.append([i, j, result.inlier_rmse, result.fitness, result.transformation])

0 0:	 Inlier RMSE - 0.00956992530017373	 Fitness: 0.19115248326677542
0 1:	 Inlier RMSE - 0.010042851986663812	 Fitness: 0.2972889363856235
0 2:	 Inlier RMSE - 0.009603991569782753	 Fitness: 0.23865234265144272
0 3:	 Inlier RMSE - 0.010855564940830226	 Fitness: 0.20040497215816414
0 4:	 Inlier RMSE - 0.011890449848003625	 Fitness: 0.11707632600258733
1 0:	 Inlier RMSE - 0.005571396400631606	 Fitness: 0.2745846261635472
1 1:	 Inlier RMSE - 0.009379070366875228	 Fitness: 0.30339805825242716
1 2:	 Inlier RMSE - 0.010804576152020602	 Fitness: 0.2514888399559604
1 3:	 Inlier RMSE - 0.01144006130007466	 Fitness: 0.1771969772795516
1 4:	 Inlier RMSE - 0.006187853787376651	 Fitness: 0.2878215393854469
2 0:	 Inlier RMSE - 0.005506929119386676	 Fitness: 0.18635036276605352
2 1:	 Inlier RMSE - 0.006156314839654624	 Fitness: 0.22525363518680194
2 2:	 Inlier RMSE - 0.01186400020931146	 Fitness: 0.12376192913267303
2 3:	 Inlier RMSE - 0.006499498654324729	 Fitness: 0.27185627841165666
2 4:	 Inlier R

In [24]:
cur_source_slide_pcd = deepcopy(source_sliding_window[5][0])
cur_target_slide_pcd = deepcopy(target_sliding_window[1][0])

cur_target_slide_pcd = cur_target_slide_pcd.transform(results[26][4])

o3d.visualization.draw_geometries([cur_source_slide_pcd, cur_target_slide_pcd])

cur_source_slide_pcd = deepcopy(source_sliding_window[5][0])
cur_target_slide_pcd = deepcopy(target_sliding_window[2][0])

cur_target_slide_pcd = cur_target_slide_pcd.transform(results[27][4])

o3d.visualization.draw_geometries([cur_source_slide_pcd, cur_target_slide_pcd])


In [10]:
results2 = []


def register_point_clouds_new(source, target, voxel_size = 0.05, global_dist_factor = 1.5, local_dist_factor = 0.4):
    source_down, source_fpfh = downsample_and_compute_fpfh(source, voxel_size)
    target_down, target_fpfh = downsample_and_compute_fpfh(target, voxel_size)

    distance_threshold = voxel_size * global_dist_factor
    # print(":: RANSAC registration on downsampled point clouds.")
    # print("   Since the downsampling voxel size is %.3f," % voxel_size)
    # print("   we use a liberal distance threshold %.3f." % distance_threshold)
    result_ransac = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh, True,
        distance_threshold,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        3, [
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(
                0.9),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(
                distance_threshold)
        ], o3d.pipelines.registration.RANSACConvergenceCriteria(400, 120))

    # Refine the registration using ICP
    result_icp = o3d.pipelines.registration.registration_icp(
        source_down, target_down, voxel_size*local_dist_factor, result_ransac.transformation,
        o3d.pipelines.registration.TransformationEstimationPointToPoint()
    )

    return result_icp

for i, (source_slide_pcd, source_slide_pose) in enumerate(source_sliding_window):
    for j, (target_slide_pcd, target_slide_pose) in enumerate(target_sliding_window):

        cur_source_slide_pcd = deepcopy(source_slide_pcd)
        cur_target_slide_pcd = deepcopy(target_slide_pcd)

        # o3d.visualization.draw_geometries([cur_source_slide_pcd, cur_target_slide_pcd])

        # mean centering
        cur_source_slide_pcd = cur_source_slide_pcd.translate(-source_slide_pose)
        cur_target_slide_pcd = cur_target_slide_pcd.translate(-target_slide_pose)

        # registration

        # o3d.visualization.draw_geometries([cur_source_slide_pcd, cur_target_slide_pcd])

        result = register_point_clouds_new(cur_source_slide_pcd, cur_target_slide_pcd)

        print(f"{i} {j}:\t Inlier RMSE - {result.inlier_rmse}\t Fitness: {result.fitness}")

        cur_target_slide_pcd = cur_target_slide_pcd.transform(result.transformation)

        # o3d.visualization.draw_geometries([cur_source_slide_pcd, cur_target_slide_pcd])

        results2.append([i, j, result.inlier_rmse, result.fitness, result.transformation])

0 0:	 Inlier RMSE - 0.008019880139090177	 Fitness: 0.00227796838967321
0 1:	 Inlier RMSE - 0.012194382262289128	 Fitness: 0.05866471680071995
0 2:	 Inlier RMSE - 0.010956446333373605	 Fitness: 0.036981832499015696
0 3:	 Inlier RMSE - 0.011391774170697028	 Fitness: 0.014399010068057822
0 4:	 Inlier RMSE - 0.0	 Fitness: 0.0
1 0:	 Inlier RMSE - 0.011246237459574756	 Fitness: 0.007143929536582925
1 1:	 Inlier RMSE - 0.011182181441408418	 Fitness: 0.002790011009908918
1 2:	 Inlier RMSE - 0.012375891321438145	 Fitness: 0.02592333099789811
1 3:	 Inlier RMSE - 0.011512315631132567	 Fitness: 0.02121909718746872
1 4:	 Inlier RMSE - 0.011021785546642685	 Fitness: 0.010809728755880292
2 0:	 Inlier RMSE - 0.01180457025511419	 Fitness: 0.002754613601469127
2 1:	 Inlier RMSE - 0.011374038478191123	 Fitness: 0.012719390673450341
2 2:	 Inlier RMSE - 0.012544928654171299	 Fitness: 0.007737002137459734
2 3:	 Inlier RMSE - 0.01142240005319002	 Fitness: 0.011417346539422585
2 4:	 Inlier RMSE - 0.0111878125

In [25]:
cur_source_slide_pcd = deepcopy(source_sliding_window[5][0])
cur_target_slide_pcd = deepcopy(target_sliding_window[1][0])

cur_target_slide_pcd = cur_target_slide_pcd.transform(results2[26][4])

o3d.visualization.draw_geometries([cur_source_slide_pcd, cur_target_slide_pcd])

cur_source_slide_pcd = deepcopy(source_sliding_window[5][0])
cur_target_slide_pcd = deepcopy(target_sliding_window[2][0])

cur_target_slide_pcd = cur_target_slide_pcd.transform(results2[27][4])

o3d.visualization.draw_geometries([cur_source_slide_pcd, cur_target_slide_pcd])


In [28]:
results_copy = deepcopy(results)
results2_copy = deepcopy(results2)

In [29]:
import pickle as pk

In [30]:
# with open("results.pkl", "wb") as f:
#     pk.dump(results, f)

In [31]:
# with open("results2.pkl", "wb") as f:
#     pk.dump(results2, f)

In [59]:
def do_it(results):
    max_fitness, max_fitness_ind = -1, -1

    for idx, (i, j, inlier_rmse, fitness, transformation) in enumerate(results):
        if fitness > max_fitness:
            max_fitness = fitness
            max_fitness_ind = idx
            

    i, j, inlier_rmse, fitness, transformation = results[max_fitness_ind]

    print(f"Using source sliding window {i} and target sliding window {j} with fitness {fitness}")

    cur_source_slide_pcd = deepcopy(source_sliding_window[i][0])
    cur_target_slide_pcd = deepcopy(target_sliding_window[j][0])

    cur_target_slide_pcd = cur_target_slide_pcd.transform(transformation)

    o3d.visualization.draw_geometries([cur_source_slide_pcd, cur_target_slide_pcd])

    source_pcd_copy = deepcopy(source_pcd)
    target_pcd_copy = deepcopy(target_pcd)

    source_pcd_copy.translate(-source_sliding_window[i][1])    
    target_pcd_copy.translate(-target_sliding_window[j][1])

    target_pcd_copy = target_pcd_copy.transform(transformation)

    o3d.visualization.draw_geometries([source_pcd_copy, target_pcd_copy])

In [61]:
do_it(results)

Using source sliding window 5 and target sliding window 2 with fitness 0.6623649368437072


In [56]:
source_pcd_copy = deepcopy(source_pcd)
target_pcd_copy = deepcopy(target_pcd)

target_pcd_copy.translate([10, 10, 10])

o3d.visualization.draw_geometries([target_pcd_copy])